# Week 10 Problem 3

A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `YOUR CODE HERE`. Do not write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select *Kernel*, and restart the kernel and run all cells (*Restart & Run all*).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select *File* → *Save and CheckPoint*)

5. When you are ready to submit your assignment, go to *Dashboard* → *Assignments* and click the *Submit* button. Your work is not submitted until you click *Submit*.

6. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

7. **If your code does not pass the unit tests, it will not pass the autograder.**

# Due Date: 6 PM, April 02, 2018

## Author: Kelechi Ikegwu
### Primary Reviewer:  John Nguyen

In [21]:
from nose.tools import assert_equal, assert_true, assert_false, assert_almost_equal
import numpy.testing as npt

from nltk.corpus import wordnet as wn
#import gensim.downloader as api

import pandas as pd
import numpy as np
from collections import Counter
import gensim
from gensim import corpora, models
from gensim.models import Word2Vec

### Problem 3.1
Write a function called *get_path_similarity* that takes in two words and calculates their path similarity using the wordnet corpus. Recall  that words passed to wordnet have an ending indicated their part of speech. In this case, we will use words that are marked with *.n.01*



In [22]:
def get_path_similarity(x,y):
    """

    Parameters
    ----------

    x: the first word
    y: the second word
    
    Returns
    -------

    the path similarity between the two words
    """

    # YOUR CODE HERE
    x = wn.synset(x + '.n.01')
    y = wn.synset(y + '.n.01')
    
    return wn.path_similarity(x,y)
    


In [23]:
assert_almost_equal(get_path_similarity('dog','boy'),0.1428, 3)
assert_almost_equal(get_path_similarity('dog','friend'),0.1666, 3)
assert_almost_equal(get_path_similarity('drive','boy'),0.0833, 3)
assert_almost_equal(get_path_similarity('man','boy'),0.3333, 3)

The cell below contains tweets that you will work with in Problem 3.2 & 3.3.

In [24]:
twitter_data = [
    "I love when vendors start conversations with fictional problems that #DataScientist have.... For instance: The biggest problem data scientists have today is they have all these variables and don't know what's important. No.  That's not really a problem to figure out at all.",
    "In honor of #IWD2018, I'm going to take some time over the next ~24 hours to recognize some of the brilliant women in tech / code / data science",
    'Ooohh! Among several other nice additions in the latest matplotlib release, there is now a new "cividis" colormap that "should look effectively identical to colorblind and non-colorblind users." #Python #dataviz #DataScience',
    'Google Makes Its #AI and #MachineLearning Courses Available to All via @Strat_AI - http://klou.tt/11oqiabgtf3si #ArtificialIntelligence #Analytics #BigData #DataScience #FinTech',
    'Keras usage is most dominant in the industry (both large companies and startups) and in the overall data science community. In the research community, Keras has a much smaller market share',
    '10 Most in demand #AI Skills are #ML #Python #R #DataScience #Hadoop #BigData #Java #DataMining #Spark #SAS  https://www.techrepublic.com/article/here-are-the-10-most-in-demand-ai-skills-and-how-to-develop-them/ … HT @TechRepublic @CyraIoT',
    'Monsanto is working with an A.I. software startup to power its agricultural data science http://on.forbes.com/6018DTTyY',
    '#InternetOfThings is transforming the way #enterprises, #governments and #consumers interact w/ the physical world... buy how does #IoT work?  #AI #SmartHome #Wearables #ConnectedCar #IIoT #Digital #BigData #DataAnalytics #DataScience #IoE @MikeQuindazzi via #PwC '
    'Data-driven companies will be the winners in the digital economy. https://buff.ly/2ETCQo8  @Jabil #DataScience #businessintelligence #DigitalTransformation',
    '13 applications for #ArtificialIntelligence in #Insurance. hashtags #AI #IoT #Insurtech #Fintech #BigData #DataScience link http://bit.ly/2F4trhN',
    'This Is Who Controls Bitcoin https://www.forbes.com/sites/ktorpey/2018/02/28/this-is-who-controls-bitcoin/#5cda221f7bc9 … #blockchain #cryptocurrency #bitcoin #litecoin #neo #decentralizedblockchain #blockchains #Crypto #datascience #cryptocurrencynews #CryptocurrencyExchange #datascience #ethereum',
    'How Millennials Are Using Cryptocurrency To Build The Future https://www.forbes.com/sites/laurencebradford/2018/02/26/how-millennials-are-using-cryptocurrency-to-build-the-future/#59e6f53d5c3a … #blockchain #cryptocurrency #bitcoin #litecoin #neo #decentralizedblockchain #blockchains #Crypto #datascience #cryptocurrencynews #CryptocurrencyExchange #datascience #ethereum',
    '#AI Helps Identify People at Risk for Suicide https://www.wsj.com/articles/ai-helps-identify-people-at-risk-for-suicide-1519400853 … #DataScience #MachineLearning',
    'By 2021, expenditures for #AI (#artificialintelligence automation) + #RPA (#robotic process #automation) expected to reach $4.9 billion. #BigData #DataMining #NLP #MachineLearning #DeepLearning #DataScience #Software @botscamp',
    '7 More Steps to Mastering #MachineLearning With #Python | #ML #bigdata #analytics #AI #DL #datascience #datascientist @kdnuggets @ThePSF http://buff.ly/2EQ0mm5',
    "isn't data science fun!",
    "What Is #MachineLearning As A Service (MLaaS)? #bigdata #IoT #technology #datascience #AI #BI",
    "Los Angeles Teachers Give Students a Jump-Start on Data Science Careers",
    "I'll be speaking about why you can’t do data science in a GUI at... ",
    "Learning to Trade with Reinforcement Learning | Open Data Science http://bit.ly/2EXwaFy  #MachineLearning #DataScience #reinforcementlearning #NeuralNetworks #ArtificialIntelligence #ai #stocks",
]

### Problem 3.2 

Complete the function *grab_tokens* by:

1. reading in stop words from  a file. The path to the file is contained in the *stop_words_path* function parameter. [*HINT:* I'd recommend storing the words in a list to make the rest of the steps easier to work with.]
2. Next convert the 2D list of the tweets *(that are in the function parameter data)* into a 2D list of the words. i.e [["This is a tweet"]] --> [["this", "is", "a", "tweet"]]. 
3. After this process use the results from item 1 to remove the stop words from item 2.
4. Then create a new variable that contains words that appear at least 2 times from item 3. [*HINT*: You may find Counter from collections useful.]
5. Return the results from item 3 and item 4


In [39]:
def grab_tokens(data, stop_words_path):
    '''
    Parameters
    ----------

    data: list containing tweets about datascience
    stop_words_path: path to file containing stop words.
    
    Returns
    -------

    a nested list of list containing words that appear twice
    a nested list of list containing all lowercase words with stop words removed
    '''
    # YOUR CODE HERE
    f = open(stop_words_path, 'r')
    stop_words = f.read().split()
    f.close()
    
    txts = [[word for word in sentance.lower().split() if word not in stop_words] for sentance in data]
    
    frequency = Counter([word for txt in txts for word in txt])
    
    tokens = [[token for token in txt if frequency[token] >= 2] for txt in txts]
    
    return tokens, txts

In [40]:
tokens, tweets2words = grab_tokens(twitter_data, '/home/data_scientist/data/misc/english.txt')

In [41]:
assert_equal(tokens, [['#datascientist', 'problem', 'data', 'problem'], ['data', 'science'], ['#python', '#datascience'], ['#ai', '#machinelearning', 'via', '#artificialintelligence', '#analytics', '#bigdata', '#datascience', '#fintech'], ['keras', 'companies', 'data', 'science', 'keras'], ['#ai', '#ml', '#python', '#datascience', '#bigdata', '#datamining', '…'], ['data', 'science'], ['#iot', '#ai', '#bigdata', '#datascience', 'via', 'companies', '#datascience'], ['#artificialintelligence', '#ai', '#iot', '#fintech', '#bigdata', '#datascience'], ['…', '#blockchain', '#cryptocurrency', '#bitcoin', '#litecoin', '#neo', '#decentralizedblockchain', '#blockchains', '#crypto', '#datascience', '#cryptocurrencynews', '#cryptocurrencyexchange', '#datascience', '#ethereum'], ['…', '#blockchain', '#cryptocurrency', '#bitcoin', '#litecoin', '#neo', '#decentralizedblockchain', '#blockchains', '#crypto', '#datascience', '#cryptocurrencynews', '#cryptocurrencyexchange', '#datascience', '#ethereum'], ['#ai', '…', '#datascience', '#machinelearning'], ['#ai', '#bigdata', '#datamining', '#machinelearning', '#datascience'], ['#machinelearning', '#python', '#ml', '#bigdata', '#analytics', '#ai', '#datascience', '#datascientist'], ['data', 'science'], ['#machinelearning', '#bigdata', '#iot', '#datascience', '#ai'], ['data', 'science'], ['data', 'science'], ['learning', 'learning', 'data', 'science', '#machinelearning', '#datascience', '#artificialintelligence', '#ai']])

assert_equal(tweets2words, [['love', 'vendors', 'start', 'conversations', 'fictional', 'problems', '#datascientist', 'have....', 'instance:', 'biggest', 'problem', 'data', 'scientists', 'today', 'variables', 'know', 'important.', 'no.', 'really', 'problem', 'figure', 'all.'], ['honor', '#iwd2018,', 'going', 'take', 'time', 'next', '~24', 'hours', 'recognize', 'brilliant', 'women', 'tech', 'code', 'data', 'science'], ['ooohh!', 'among', 'several', 'nice', 'additions', 'latest', 'matplotlib', 'release,', 'now', 'new', '"cividis"', 'colormap', '"should', 'look', 'effectively', 'identical', 'colorblind', 'non-colorblind', 'users."', '#python', '#dataviz', '#datascience'], ['google', 'makes', '#ai', '#machinelearning', 'courses', 'available', 'via', '@strat_ai', '-', 'http://klou.tt/11oqiabgtf3si', '#artificialintelligence', '#analytics', '#bigdata', '#datascience', '#fintech'], ['keras', 'usage', 'dominant', 'industry', '(both', 'large', 'companies', 'startups)', 'overall', 'data', 'science', 'community.', 'research', 'community,', 'keras', 'much', 'smaller', 'market', 'share'], ['10', 'demand', '#ai', 'skills', '#ml', '#python', '#r', '#datascience', '#hadoop', '#bigdata', '#java', '#datamining', '#spark', '#sas', 'https://www.techrepublic.com/article/here-are-the-10-most-in-demand-ai-skills-and-how-to-develop-them/', '…', 'ht', '@techrepublic', '@cyraiot'], ['monsanto', 'working', 'a.i.', 'software', 'startup', 'power', 'agricultural', 'data', 'science', 'http://on.forbes.com/6018dttyy'], ['#internetofthings', 'transforming', 'way', '#enterprises,', '#governments', '#consumers', 'interact', 'w/', 'physical', 'world...', 'buy', '#iot', 'work?', '#ai', '#smarthome', '#wearables', '#connectedcar', '#iiot', '#digital', '#bigdata', '#dataanalytics', '#datascience', '#ioe', '@mikequindazzi', 'via', '#pwc', 'data-driven', 'companies', 'will', 'winners', 'digital', 'economy.', 'https://buff.ly/2etcqo8', '@jabil', '#datascience', '#businessintelligence', '#digitaltransformation'], ['13', 'applications', '#artificialintelligence', '#insurance.', 'hashtags', '#ai', '#iot', '#insurtech', '#fintech', '#bigdata', '#datascience', 'link', 'http://bit.ly/2f4trhn'], ['controls', 'bitcoin', 'https://www.forbes.com/sites/ktorpey/2018/02/28/this-is-who-controls-bitcoin/#5cda221f7bc9', '…', '#blockchain', '#cryptocurrency', '#bitcoin', '#litecoin', '#neo', '#decentralizedblockchain', '#blockchains', '#crypto', '#datascience', '#cryptocurrencynews', '#cryptocurrencyexchange', '#datascience', '#ethereum'], ['millennials', 'using', 'cryptocurrency', 'build', 'future', 'https://www.forbes.com/sites/laurencebradford/2018/02/26/how-millennials-are-using-cryptocurrency-to-build-the-future/#59e6f53d5c3a', '…', '#blockchain', '#cryptocurrency', '#bitcoin', '#litecoin', '#neo', '#decentralizedblockchain', '#blockchains', '#crypto', '#datascience', '#cryptocurrencynews', '#cryptocurrencyexchange', '#datascience', '#ethereum'], ['#ai', 'helps', 'identify', 'people', 'risk', 'suicide', 'https://www.wsj.com/articles/ai-helps-identify-people-at-risk-for-suicide-1519400853', '…', '#datascience', '#machinelearning'], ['2021,', 'expenditures', '#ai', '(#artificialintelligence', 'automation)', '+', '#rpa', '(#robotic', 'process', '#automation)', 'expected', 'reach', '$4.9', 'billion.', '#bigdata', '#datamining', '#nlp', '#machinelearning', '#deeplearning', '#datascience', '#software', '@botscamp'], ['7', 'steps', 'mastering', '#machinelearning', '#python', '#ml', '#bigdata', '#analytics', '#ai', '#dl', '#datascience', '#datascientist', '@kdnuggets', '@thepsf', 'http://buff.ly/2eq0mm5'], ['data', 'science', 'fun!'], ['#machinelearning', 'service', '(mlaas)?', '#bigdata', '#iot', '#technology', '#datascience', '#ai', '#bi'], ['los', 'angeles', 'teachers', 'give', 'students', 'jump-start', 'data', 'science', 'careers'], ['speaking', 'can’t', 'data', 'science', 'gui', 'at...'], ['learning', 'trade', 'reinforcement', 'learning', 'open', 'data', 'science', 'http://bit.ly/2exwafy', '#machinelearning', '#datascience', '#reinforcementlearning', '#neuralnetworks', '#artificialintelligence', '#ai', '#stocks']])


### Problem 3.3 

Complete the function *word2vec* by:
- creating a [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) model with gensim.
- Set the max distance between the current and predicted word within a sentence to be the *window* function paramater and ignore all words with a total frequency less than the *min_count* function parameter. 
- Return the Word2Vec model.

In [42]:
def word2vec(tweets2words, window, min_count):
    '''
    Parameters
    ----------

    tweets2words: a nested list of list containing all lowercase words with stop words removed
    window: The maximum distance between the current and predicted word within a sentence.
    min_count:Ignores all words with total frequency lower than this.
    
    Returns
    -------

    Word2Vec Model of type (gensim.models.word2vec.Word2Vec)
    '''
    # YOUR CODE HERE
    w2v = Word2Vec(tweets2words, window=window, min_count=min_count)
    
    return w2v
    

In [43]:
model = word2vec(tweets2words, window=2, min_count=2)
assert_true(isinstance(model, gensim.models.word2vec.Word2Vec))
assert_equal(model.window, 2)
assert_equal(model.min_count, 2)

Let us take a look at the 5 most similar words using the Word2Vec model you generated i Problem 3.3 for a few words 

In [44]:
a = model.most_similar('data', topn=5)
b = model.most_similar('problem', topn=5)

print('Top 5 most similar words for: data')
print('-'*10)
for item in a:
    print(item[0],  item[1])

print('\n')
print('Top 5 most similar words for: problem')
print('-'*10)
for item in b:
    print(item[0],  item[1])



Top 5 most similar words for: data
----------
via 0.20329558849334717
#crypto 0.14181318879127502
#bitcoin 0.09167920053005219
… 0.07954660058021545
#python 0.07488829642534256


Top 5 most similar words for: problem
----------
#cryptocurrencynews 0.13341009616851807
#crypto 0.11832500249147415
#python 0.1025846004486084
#blockchain 0.09190668910741806
#neo 0.07962703704833984
